Setup

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import torch
import CNN

# from config_plots import global_settings
# global_settings()
import warnings
warnings.filterwarnings("ignore")

Hyper parameters

In [2]:
# defining global variables
experimentName = "test_22987_suffix100"
showListOfSpecies = False

from configParser import ConfigParser, getModelName
config_parser = ConfigParser(experimentName)
    
experimentName = experimentName + "/multi-trial"
import TrialStatistics
ts = TrialStatistics.TrialStatistics(experimentName)

Use_old_dataLoader = False
if Use_old_dataLoader:
    import dataLoader
else:
    import dataLoader2 as dataLoader

Cuda support

In [3]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


Iterate

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from confusion_matrix_plotter import plot_confusion_matrix2, generate_classification_report
import numpy as np
import os
import progressbar

paramsIterator = config_parser.getHyperpIter()    
number_of_experiments = sum(1 for e in paramsIterator)
paramsIterator = config_parser.getHyperpIter()  
experiment_index = 0

with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
    for experiment_params in paramsIterator:
        bar.update(experiment_index)
        experiment_params["numOfTrials"] = experiment_params["numOfTrials"] or 1

        print("experiment ", experiment_index, "/", number_of_experiments, ": ", experiment_params)

        # load images
        dataset = dataLoader.FishDataset(experiment_params, showListOfSpecies)
        speciesList = dataset.getSpeciesList()
        numberOfSpecies = len(speciesList)

        confusionMatricesPerTrial = []

        for i in range(experiment_params["numOfTrials"]):
            trialName = experimentName+"/"+getModelName(experiment_params, i)

            # Train/Load model
            model = CNN.CNN(numberOfSpecies, experiment_params)
            if os.path.exists(CNN.getModelFile(trialName)):
                _, _, epochs, time_elapsed = CNN.loadModel(model, trialName)
                test_loader = dataLoader.loadTestLoader(trialName)
                print("Model {0} loaded!".format(trialName))
            else:
                train_loader, validation_loader, test_loader = dataLoader.getLoadersFromDataset(dataset, experiment_params, trialName)
                _, _, epochs, time_elapsed = CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName)

            # Update trial outcomes for statistics
            predlist, lbllist = CNN.getLoaderPredictions(test_loader, model)
            ts.addTrialPredictions(experiment_params, predlist, lbllist, numberOfSpecies)

            ts.addTrial(experiment_params,
                    {'loss': CNN.getCrossEntropyFromLoader(test_loader, model),
                     'accuracy': CNN.getAccuracyFromLoader(test_loader, model),
                     'time': time_elapsed,
                     'epochs': epochs
                    }, i)
        
        experiment_index = experiment_index + 1

N/A% (0 of 24) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

experiment  0 / 24 :  {'image_path': 'INHS_cropped', 'suffix': 100, 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 1000, 'numOfTrials': 3, 'kernels': [16, 32], 'kernelSize': 5, 'patience': 20, 'imageDimension': 28, 'n_channels': 1, 'useZCAWhitening': False}
Loading dataset...


N/A% (0 of 24) |                         | Elapsed Time: 0:00:01 ETA:  --:--:--

Loading saved dataset structure...


100% (3800 of 3800) |####################| Elapsed Time: 0:00:29 Time:  0:00:29


...

100% (1000 of 1000) |####################| Elapsed Time: 4:10:48 Time:  4:10:48


Early stopping
total number of epochs:  234


N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training started...


  3% (30 of 1000) |                      | Elapsed Time: 0:32:10 ETA:  17:15:33

Analyze

In [ ]:
ts.showStatistics()
ts.showStatistics(False)
ts.saveStatistics()
ts.saveStatistics(False)

paramsIterator = config_parser.getHyperpIter() 
for experiment_params in paramsIterator:
    experiment_params["numOfTrials"] = experiment_params["numOfTrials"] or 1
    
    print("experiment: ", experiment_params)
    
    dataset = dataLoader.FishDataset(experiment_params, showListOfSpecies)
    speciesList = dataset.getSpeciesList()
    ts.printTrialConfusionMatrix(experiment_params, speciesList, True)